In [2]:
### Load library -----
library(DESeq2)
library(tidyverse)

In [3]:
### import GO DB -----
#Import GO database
GOdata<-read.csv('Reference csv/Cleaned_GO_4.csv', sep=';', header=FALSE)
GOdata$Gene<-as.character(GOdata$V1)
GOdata$V1<-NULL
GOdata$geneabb<-as.character(GOdata$V2)
GOdata$V2<-NULL
GOdata$genename<-as.character(GOdata$V3)
GOdata$V3<-NULL
GOdata$GO<-as.character(GOdata$V4)
GOdata$V4<-NULL

### featureCount data import -----
#Importing output of featurecounts
countdata <- read.table("featureCounts/01.DMSO_Maya2_WT_Leaf_new", header=TRUE, row.names='Geneid' )

#remove first 5 lines which is useless
countdata <- countdata[ ,6:ncol(countdata)]

#remove .bam in the sample name
colnames(countdata) <- gsub("\\_a.bam$", "", colnames(countdata))
rownames(countdata) <- gsub("gene:", "", rownames(countdata))
colnames(countdata)

#Import sample information
saminfo <- read.csv("Saminfo/01.Saminfo_DMSO_Maya2_Leaf_new.csv")

#change column names using saminfo
colnames(countdata) <- saminfo$condition
colnames(countdata)

#sort by column names
saminfo <- arrange(saminfo, index)
countdata <- countdata[,c(saminfo$index)]

#change it to matrix
countdata <- as.matrix(countdata)
head(countdata)

[1] "X183397_10"      "X183398_11"      "X183399_12"      "X183403_16"     
 [5] "X183404_17"      "X183405_18"      "X183412_25"      "X183413_26"     
 [9] "X183414_27"      "X183418_31_.sam" "X183419_32_.sam" "X183420_33_.sam"

[1] "X183397_10" "X183398_11" "X183399_12" "X183403_16" "X183404_17"
 [6] "X183405_18" "X183412_25" "X183413_26" "X183414_27" "X183418_31"
[11] "X183419_32" "X183420_33"

,X183397_10,X183398_11,X183399_12,X183403_16,X183404_17,X183405_18,X183412_25,X183413_26,X183414_27,X183418_31,X183419_32,X183420_33
AT1G01010,18,20,15,23,29,16,123,124,69,50,39,31
AT1G01020,85,127,110,98,103,63,169,135,86,64,103,97
AT1G03987,4,2,0,0,0,0,0,0,1,0,1,0
AT1G01030,32,9,45,17,10,11,202,242,147,61,125,61
AT1G01040,110,241,184,136,126,83,208,215,183,139,113,109
AT1G03993,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
### DEseq analysis -----
#define factor for DEseq
condition <- factor(c(rep('DMSO1_WT', 3), rep('DMSO1_fls2', 3), rep('Maya2_WT', 3), rep('Maya2_fls2', 3)))

saminfo$colnames_count <- colnames(countdata)
coldata <- data.frame(row.names=colnames(countdata), condition)

#Chem screening
dds <- DESeqDataSetFromMatrix(countData=countdata, colData=coldata, design=~condition)
dds <- DESeq(dds)
colnames(dds) <- condition

res_wt <- results(dds, contrast=c('condition', 'Maya2_WT', 'DMSO1_WT'))
res_fls2 <- results(dds, contrast=c('condition', 'Maya2_fls2', 'DMSO1_fls2'))

table(res_wt$padj<0.1)
table(res_fls2$padj<0.1)

resDF_wt<-as.data.frame(res_wt)
resDF_fls2<-as.data.frame(res_fls2)

output_wt <- merge(resDF_wt, as.data.frame(counts(dds, normalized=TRUE)), by="row.names", sort=FALSE)
output_fls2 <- merge(resDF_fls2, as.data.frame(counts(dds, normalized=TRUE)), by="row.names", sort=FALSE)

names(output_wt)[1] <- "Gene"
names(output_fls2)[1] <- "Gene"

output_wt<-merge(GOdata, output_wt, by="Gene")
output_fls2<-merge(GOdata, output_fls2, by="Gene")

write.csv(output_wt, file="DESeq2/Original/DEseq2_results_WT_Maya2_leaf.csv")
write.csv(output_fls2, file="DESeq2/Original/DEseq2_results_fls2_Maya2_leaf.csv")


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing




FALSE  TRUE 
14395  3543 


FALSE  TRUE 
14978  2521 

In [6]:
### Generate final table -----
Total <- merge(output_wt, output_fls2, by = "Gene")

colnames(Total)
Total <- Total %>%
  dplyr::select(Gene, geneabb.x, genename.x, log2FoldChange.x, padj.x,
                log2FoldChange.y, padj.y,
                starts_with("DMSO_WT"), starts_with("Maya2_WT"),
                starts_with("DMSO_fls2"), starts_with("Maya2_fls2"),
                GO.x)
colnames(Total)

colnames(Total) <- c("Gene", "Gene_Symbol", "Gene_Name", 
                     "Log2FC_WT", "Padj_WT", 
                     "Log2FC_fls2", "Padj_fls2", 
                     "DMSO_WT_rep1", "DMSO_WT_rep2", "DMSO_WT_rep3",
                     "Maya2_WT_rep1", "Maya2_WT_rep2", "Maya2_WT_rep3",                     
                     "DMSO_fls2_rep1", "DMSO_fls2_rep2", "DMSO_fls2_rep3", 
                     "Maya2_fls2_rep1", "Maya2_fls2_rep2", "Maya2_fls2_rep3", 
                     "GO")
colnames(Total)

write.csv(Total, file="Total/Total_Maya2_leaf.csv")


[1] "Gene"             "geneabb.x"        "genename.x"       "GO.x"            
 [5] "baseMean.x"       "log2FoldChange.x" "lfcSE.x"          "stat.x"          
 [9] "pvalue.x"         "padj.x"           "DMSO1_WT.x"       "DMSO1_WT.1.x"    
[13] "DMSO1_WT.2.x"     "DMSO1_fls2.x"     "DMSO1_fls2.1.x"   "DMSO1_fls2.2.x"  
[17] "Maya2_WT.x"       "Maya2_WT.1.x"     "Maya2_WT.2.x"     "Maya2_fls2.x"    
[21] "Maya2_fls2.1.x"   "Maya2_fls2.2.x"   "geneabb.y"        "genename.y"      
[25] "GO.y"             "baseMean.y"       "log2FoldChange.y" "lfcSE.y"         
[29] "stat.y"           "pvalue.y"         "padj.y"           "DMSO1_WT.y"      
[33] "DMSO1_WT.1.y"     "DMSO1_WT.2.y"     "DMSO1_fls2.y"     "DMSO1_fls2.1.y"  
[37] "DMSO1_fls2.2.y"   "Maya2_WT.y"       "Maya2_WT.1.y"     "Maya2_WT.2.y"    
[41] "Maya2_fls2.y"     "Maya2_fls2.1.y"   "Maya2_fls2.2.y"

[1] "Gene"             "geneabb.x"        "genename.x"       "log2FoldChange.x"
 [5] "padj.x"           "log2FoldChange.y" "padj.y"           "Maya2_WT.x"      
 [9] "Maya2_WT.1.x"     "Maya2_WT.2.x"     "Maya2_WT.y"       "Maya2_WT.1.y"    
[13] "Maya2_WT.2.y"     "Maya2_fls2.x"     "Maya2_fls2.1.x"   "Maya2_fls2.2.x"  
[17] "Maya2_fls2.y"     "Maya2_fls2.1.y"   "Maya2_fls2.2.y"   "GO.x"

[1] "Gene"            "Gene_Symbol"     "Gene_Name"       "Log2FC_WT"      
 [5] "Padj_WT"         "Log2FC_fls2"     "Padj_fls2"       "DMSO_WT_rep1"   
 [9] "DMSO_WT_rep2"    "DMSO_WT_rep3"    "Maya2_WT_rep1"   "Maya2_WT_rep2"  
[13] "Maya2_WT_rep3"   "DMSO_fls2_rep1"  "DMSO_fls2_rep2"  "DMSO_fls2_rep3" 
[17] "Maya2_fls2_rep1" "Maya2_fls2_rep2" "Maya2_fls2_rep3" "GO"